In [1]:
from sqlalchemy import create_engine
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json

# Staging
server_stg = '34.95.134.241'
database_stg = 'VoistonDb'
username_stg = 'sqlserver'
password_stg = '.KN_mWXA7kxsnat'

# Prod
server_prod = '34.95.157.73'
database_prod = 'VoistonDb'
username_prod = 'sqlserver'
password_prod = 'BF2y4d!.DqR6'
# String de conexão com SQLAlchemy

engine_stg = create_engine(f"mssql+pyodbc://{username_stg}:{password_stg}@{server_stg}/{database_stg}?driver=ODBC+Driver+17+for+SQL+Server")
engine_prod = create_engine(f"mssql+pyodbc://{username_prod}:{password_prod}@{server_prod}/{database_prod}?driver=ODBC+Driver+17+for+SQL+Server")

In [ ]:
# Codigo para pegar os PartnerIDs de stg e entao ler em prod:
# patient_ids_list1 = [
#     280530, 280543, 280544, 280547, 280549, 280550, 280551, 280554, 280555, 
#     280545, 280546, 280548, 280552, 280553, 280556, 280557, 280558, 280559, 
#     280560, 280561, 280562, 280563, 280564, 280565, 280566, 280567, 280568, 
#     280569, 280570, 280571, 280531
# ]

# patient_ids_list2 = [
#     280524, 280525, 280526, 280527, 280528, 280529,
#     280530, 280531, 280532, 280533, 280534, 280535,
#     280536, 280537, 280538, 280539, 280540, 280541,
#     280542, 280543, 280544, 280545, 280546, 280547,
#     280548, 280549, 280550, 280551, 280552, 280553,
#     280554, 280563, 280564, 280565, 280566, 280567,
#     280568, 280569, 280570, 280571, 280572, 280573,
#     280574, 280575, 280576, 280577, 280578, 280579, 280580,
#     280581, 280582
# ]

# patient_ids_list = [280583, 280584, 280585, 280586, 280587, 280588, 280589, 280590, 280591, 280592, 280593, 280594, 280595, 280596, 280597, 280598, 280599, 280600, 280601, 280602, 280603, 280604, 280605,
# 280606,280607,280608,280609,280610,280611,280612,280613,280614,280615,280616,280617,280618,280619,280620,280621,280622,280623,280624,280625,280626,280627,280628,280629,280630,280631,280632,280633,280634,
# 280635,280636,280637,280638,280639,280640,280641,280642,280643,280644,280645,280646,280647,280648,280649,280650,280651,280652,280653,280654,280655,280656,280657,280658,280659,280660,280661,280662,280663,280664,
# 280665,280666,280667,280668,280669,280670,280671,280672,280673,280674,280675,280676,280677,280678,280679,280680,280681,280682,280683,280684,280685,280686,280687,280688,280689,280690,280691,280692,280693,280694,
# 280695,280696,280697,280698,280699,280700,280701,280702,280703,280704,280705,280706,280707,280708,280709,280710,280711,280712,280713,280714,280715,280716,280717,280718,280719,280720,280721,280722,280723,280724,
# 280725,280726,280727,280728,280729,280730,280731,280732,280733,280734,280735,280736,280737,280738,280739,280740,280741,280742,280743,280744,280745,280746,280747,280748,280749,280750,280751,280752,280753,280754,
# 280755,280756,280757,280758,280759,280760,280761,280763]

# patients_ids_sql = ", ".join(map(str, patient_ids_list))

# query_partner_ids = f"""
# SELECT DISTINCT PartnerID
# FROM EHRs e
# WHERE e.Patient_ID IN ({patient_ids_sql})
# """


In [6]:
patient_ids_sql

'280583, 280584, 280585, 280586, 280587, 280588, 280589, 280590, 280591, 280592, 280593, 280594, 280595, 280596, 280597, 280598, 280599, 280600, 280601, 280602, 280603, 280604, 280605, 280606, 280607, 280608, 280609, 280610, 280611, 280612, 280613, 280614, 280615, 280616, 280617, 280618, 280619, 280620, 280621, 280622, 280623, 280624, 280625, 280626, 280627, 280628, 280629, 280630, 280631, 280632, 280633, 280634, 280635, 280636, 280637, 280638, 280639, 280640, 280641, 280642, 280643, 280644, 280645, 280646, 280647, 280648, 280649, 280650, 280651, 280652, 280653, 280654, 280655, 280656, 280657, 280658, 280659, 280660, 280661, 280662, 280663, 280664, 280665, 280666, 280667, 280668, 280669, 280670, 280671, 280672, 280673, 280674, 280675, 280676, 280677, 280678, 280679, 280680, 280681, 280682, 280683, 280684, 280685, 280686, 280687, 280688, 280689, 280690, 280691, 280692, 280693, 280694, 280695, 280696, 280697, 280698, 280699, 280700, 280701, 280702, 280703, 280704, 280705, 280706, 280707,

In [22]:
query_patient_ids = """
SELECT
    patient_id
FROM
    ehrs
WHERE
    patient_id IN (SELECT id FROM patients WHERE name LIKE 'qareview%')
    AND processWithLLMs = 1
GROUP BY
    patient_id
HAVING
    COUNT(CASE WHEN processingStatus = 5 THEN 1 END) = 0
    AND COUNT(CASE WHEN ProcessingStatus > 5 AND LLMStepStatus < 200 THEN 1 END) = 0
"""

In [23]:
ids = pd.read_sql(query_patient_ids, engine_stg)
patients_ids_sql = ", ".join(map(str, ids['patient_id'].tolist()))

In [24]:
patients_ids_sql

'280525, 280526, 280527, 280528, 280529, 280530, 280531, 280532, 280533, 280534, 280535, 280536, 280537, 280538, 280539, 280540, 280541, 280542, 280543, 280544, 280545, 280546, 280547, 280548, 280549, 280550, 280551, 280552, 280553, 280554, 280563, 280564, 280565, 280566, 280567, 280568, 280569, 280570, 280571, 280572, 280573, 280574, 280575, 280576, 280577, 280578, 280579, 280580, 280581, 280582, 280583, 280584, 280585, 280586, 280587, 280588, 280589, 280590, 280591, 280592, 280593, 280594, 280595, 280596, 280597, 280598, 280599, 280600, 280601, 280602, 280603, 280604, 280605, 280606, 280607, 280608, 280609, 280610, 280611, 280612, 280613, 280614, 280615, 280616, 280617, 280618, 280619, 280620, 280621, 280622, 280623, 280624, 280625, 280626, 280627, 280628, 280629, 280630, 280631, 280632, 280633, 280634, 280635, 280636, 280637, 280638, 280639, 280640, 280641, 280642, 280643, 280644, 280645, 280646, 280647, 280648, 280649, 280650, 280651, 280652, 280653, 280654, 280655, 280656, 280657,

In [25]:
query_partner_ids = f"""
SELECT DISTINCT PartnerID
FROM EHRs e
WHERE e.Patient_ID IN ({patients_ids_sql})
"""


partner_ids = (
    pd.read_sql(query_partner_ids, engine_stg)["PartnerID"]
    .dropna()
    .astype(str)
    .unique()
    .tolist()
)

In [26]:
# Proximo passo: pegar os EHRs com os Contains em stg e rodar o codigo de flag bits:
FLAGS = [
    ("DiagnosisOphthalmological", 1), ("DiagnosisComorbidities", 2), ("FamiliarHistory", 4),
    ("PreviousOphthalmologicalProcedures", 8), ("MedicationsInUse", 16), ("MedicationsPrescribed", 32),
    ("SlitLamp", 64), ("NeurossensorialOcularEvaluation", 128), ("IntraocularPressure", 256),
    ("VisualAcuity", 512), ("Refraction", 1024), ("Cid", 2048), ("MedicalConduct", 4096),
    ("RefractionOnly", 8192), ("ExaminationReport", 16384), ("SurgicalDescription", 32768),
    ("AnesthesiaRecord", 65536), ("MedicalLeaveCertificate", 131072), ("MedicalReport", 262144),
    ("PreopEvaluation", 524288), ("InformedConsent", 1048576), ("PatientWish", 2097152),
    ("TreatmentContraindication", 4194304),
]
flag_names = [name for name, _ in FLAGS]

select_bits = ",\n    ".join(
    [
        f"CAST(CASE WHEN ([e].[Contains] & {val}) = {val} THEN 1 ELSE 0 END AS int) AS [{name}]"
        for name, val in FLAGS
    ]
)

query_ehrs_stg = f"""
SELECT
    p.Name AS PatientName,
    p.ID AS Patient_ID,
    e.ID AS EHR_ID,
    e.PartnerID,
    e.FormattedText,
    COALESCE(e.DateTaken, e.DateCreated) AS EHR_Date,
    e.[Contains],
    {select_bits}
FROM EHRs e
JOIN Patients p ON e.Patient_ID = p.ID
WHERE e.Patient_ID IN ({patients_ids_sql})
"""
df_ehrs = pd.read_sql(query_ehrs_stg, engine_stg)

In [27]:
df_ehrs.shape

(45033, 30)

In [28]:
ehr_flags = df_ehrs.melt(
    id_vars=["EHR_ID", "Patient_ID", "PartnerID", "EHR_Date", "FormattedText", "Contains"],
    value_vars=flag_names,
    var_name="ClassificationType",
    value_name="IsPresent",
)
ehr_flags = ehr_flags[ehr_flags["IsPresent"] == 1].drop(columns="IsPresent")
ehr_flags.head()

,EHR_ID,Patient_ID,PartnerID,EHR_Date,FormattedText,Contains,ClassificationType
0,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,DiagnosisOphthalmological
9,4640903,280525,NP:709436_APAC:2335312_OCM:1823012,2019-08-03 10:21:23,Consulta Médica\r\nID da Consulta: 1823012\r\n...,5953,DiagnosisOphthalmological
14,4640908,280525,NP:709436_APAC:3748409_OCM:3189225,2024-06-15 08:45:14,Consulta Médica\r\nID da Consulta: 3189225\r\n...,2103131,DiagnosisOphthalmological
19,4640913,280525,NP:709436_APAC:3892882_OCM:3334237,2024-11-22 09:47:10,Consulta Médica\r\nID da Consulta: 3334237\r\n...,2102107,DiagnosisOphthalmological
30,4640924,280526,NP:995878_APAC:2574555_OCM:2043543,2020-08-13 14:29:00,Consulta Médica\r\nID da Consulta: 2043543\r\n...,5953,DiagnosisOphthalmological


In [29]:
# --------------------------------------------
# 4) MEDIDAS e PRESCRIÇÕES — LLM
# --------------------------------------------
query_med_stg = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    CASE WHEN m.LLMPrompt_ID IS NULL THEN 'parser' ELSE 'llm' END AS extractionType,
    m.Type_ID AS Measurement_ID,
    m.Side,
    m.StringValue,
    m.Metadata,
    mt.Name AS MeasurementName
FROM Measurements m
JOIN EHRs e ON e.ID = m.EHR_ID
LEFT JOIN MeasurementTypes mt ON m.Type_ID = mt.ID
WHERE e.Patient_ID IN ({patients_ids_sql})
"""
df_meas_stg = pd.read_sql(query_med_stg, engine_stg)
df_meas_stg.head()

,EHR_ID,PartnerID,extractionType,Measurement_ID,Side,StringValue,Metadata,MeasurementName
0,4640897,NP:709436_APAC:1981525_OCM:1507607,llm,44,2,20/20,None,Visual Acuity Snellen
1,4640897,NP:709436_APAC:1981525_OCM:1507607,llm,44,1,20/20,None,Visual Acuity Snellen
2,4640903,NP:709436_APAC:2335312_OCM:1823012,llm,44,2,20/60,None,Visual Acuity Snellen
3,4640903,NP:709436_APAC:2335312_OCM:1823012,llm,44,1,20/20,None,Visual Acuity Snellen
4,4640908,NP:709436_APAC:3748409_OCM:3189225,llm,44,2,20/20,None,Visual Acuity Snellen


In [30]:
query_rx_stg = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    CASE WHEN m.LLMPrompt_ID IS NULL THEN 'parser' ELSE 'llm' END AS extractionType,
    m.Medication_ID AS Measurement_ID,
    m.Side,
    m.Frequency,
    m.FrequencyInstructions,
    m.Duration,
    m.StartDay,
    m.MedicationDosage,
    m.Dosage,
    m.MedicationDosageUnit,
    m.RouteOfAdministration,
    m.Metadata
FROM Prescriptions m
JOIN EHRs e ON e.ID = m.EHR_ID
WHERE e.Patient_ID IN ({patients_ids_sql})
"""
df_rx_stg = pd.read_sql(query_rx_stg, engine_stg)
df_rx_stg.head()

,EHR_ID,PartnerID,extractionType,Measurement_ID,Side,Frequency,FrequencyInstructions,Duration,StartDay,MedicationDosage,Dosage,MedicationDosageUnit,RouteOfAdministration,Metadata
0,4640971,NP:232826_APAC:3998362_MR:1844948,llm,10922,0,4,de 4 em 4 horas,7,0,0.0,1.0,0,150,None
1,4640971,NP:232826_APAC:3998362_MR:1844948,llm,14102,0,4,De 4 em 4 horas,5,0,0.0,1.0,0,150,None
2,4640926,NP:995878_APAC:2672768_MR:1039794,llm,8351,3,6,4 vezes ao dia,0,0,0.0,1.0,0,150,None
3,4641005,NP:941260_APAC:3670607_MR:1647078,llm,10922,3,4,4/4 horas,10,0,0.0,1.0,0,150,None
4,4641005,NP:941260_APAC:3670607_MR:1647078,llm,14102,3,4,4/4 horas,20,0,0.0,1.0,0,150,None


In [31]:
query_proc_stg = f"""
SELECT
    pe.EHR_ID,
    e.PartnerID,
    CASE WHEN pe.LLMPrompt_ID IS NULL THEN 'parser' ELSE 'llm' END AS extractionType,
    pe.Procedure_ID,
    pe.ProcedureType_ID,
    pt.DisplayName,
    pe.Side,
    pe.PeriodSpan,
    pe.Metadata,
    pr.ProcedureDate,
    pe.DateCreated AS EvidenceCreated
FROM ProcedureEvidences pe
JOIN EHRs e ON pe.EHR_ID = e.ID
JOIN ProcedureTypes pt ON pe.ProcedureType_ID = pt.ID
LEFT JOIN Procedures pr ON pe.Procedure_ID = pr.ID
WHERE e.Patient_ID IN ({patients_ids_sql})
"""
df_proc_stg = pd.read_sql(query_proc_stg, engine_stg)
df_proc_stg.head()

,EHR_ID,PartnerID,extractionType,Procedure_ID,ProcedureType_ID,DisplayName,Side,PeriodSpan,Metadata,ProcedureDate,EvidenceCreated
0,4640906,NP:709436_APAC:3061146_OCM:2508491,llm,2133683.0,6,FACO,0,2,None,2025-08-14 08:23:50,2025-09-10 19:53:10.820
1,4640908,NP:709436_APAC:3748409_OCM:3189225,llm,2133693.0,6,FACO,2,4,None,2020-06-16 08:45:14,2025-09-10 19:53:31.990
2,4640913,NP:709436_APAC:3892882_OCM:3334237,llm,2133693.0,6,FACO,2,4,None,2020-06-16 08:45:14,2025-09-12 01:42:36.300
3,4640918,NP:709436_APAC:4124734_C:413426_CD:232244,llm,2133683.0,6,FACO,0,0,None,2025-08-14 08:23:50,2025-09-10 19:53:37.180
4,4640918,NP:709436_APAC:4124734_C:413426_CD:232244,llm,2133687.0,51,LIO,0,0,None,2025-08-14 08:23:50,2025-09-10 19:53:37.183


In [32]:
meas_IDs = pd.read_sql("""
SELECT *
FROM MeasurementTypes
""", engine_prod)
meas_IDs.head()

,ID,Name,Unit,AlternativeNames,AnatomyElementSpecific,SideSpecific,DateCreated,LastModified,MeasurementType_ID,ScaleMax,...,LimitMin,ScaleStep,DisplayName,Priority,Metadata,DisplaySeriesAs,DisplaySeriesColor,DashboardUnit,AlertMax,AlertMin
0,41,Average RNFL Thickness,100,None,False,True,2020-01-07 20:35:15.127,2020-01-07 20:35:15.127,NaN,150.00,...,0.0,50.0,Avg RNFL Thickness,7.0000,False,0,None,0,123.5,47.5
1,42,Average Superior RNFL,100,None,False,True,2020-01-07 20:35:15.127,2020-01-07 20:35:15.127,41.0,150.00,...,0.0,50.0,Avg Sup RNFL,7.1000,False,0,None,0,159.5,43.5
2,43,Average Inferior RNFL,100,None,False,True,2020-01-07 20:35:15.127,2020-01-07 20:35:15.127,41.0,150.00,...,0.0,50.0,Avg Inf RNFL,7.2000,False,0,None,0,180.0,28.0
3,44,Visual Acuity Snellen,109,"acuidade visual com correção, acuidadevisual c...",False,True,2020-01-07 20:35:15.127,2025-08-17 02:23:43.537,NaN,1.05,...,0.0,0.1,Acuidade Visual,1.0001,False,0,None,0,2.0,0.0
4,45,Visual Acuity Snellen No Correction,109,"acuidade visual sem correção, acuidadevisual s...",False,True,2020-01-07 20:35:15.127,2025-08-17 02:25:09.560,44.0,1.05,...,0.0,0.1,Acuidade Visual (sc),1.1000,False,0,None,0,2.0,0.0


In [16]:
partner_ids

['NP:1007920_APAC:3853703_OCM:3294208',
 'NP:1007920_APAC:3938387_OCM:3379480',
 'NP:1007920_APAC:3938387_PEE:1050727',
 'NP:1007920_APAC:4049482_AP:455341',
 'NP:1008382_APAC:2719673_PEE:245340',
 'NP:1008382_APAC:3870723_PEE:1002434',
 'NP:1008382_APAC:4092274_OCM:3533096',
 'NP:1009687_APAC:2934412_OCM:2386064',
 'NP:1009687_APAC:2934412_PEE:362269',
 'NP:1009687_APAC:3086485_C:335927_CD:154505',
 'NP:1009687_APAC:3352903_C:354137_CD:172535',
 'NP:1009687_APAC:3473574_PEE:722308',
 'NP:1009687_APAC:3487730_OCM:2926481',
 'NP:1009687_APAC:3805016_PEE:955040',
 'NP:1009687_APAC:3863809_OCM:3304427',
 'NP:1009687_APAC:3864996_PEE:998077',
 'NP:1009687_APAC:3864996_PEE:998082',
 'NP:1009687_APAC:3880304_PEE:1009487',
 'NP:1009687_APAC:4113279_OCM:3555047',
 'NP:1015618_APAC:3223496_OCM:2665457',
 'NP:1015618_APAC:3290143_OCM:2730147',
 'NP:1015618_APAC:3873360_MR:1769383',
 'NP:1015618_APAC:3988832_OCM:3429967',
 'NP:1015618_APAC:3988832_OCM:3429994',
 'NP:1015618_APAC:3988832_PEE:10858

In [37]:
print("Tipo:", type(partner_ids))
print("Tamanho:", len(partner_ids))
print("Primeiros 5 itens:", partner_ids[:5])

Tipo: <class 'list'>
Tamanho: 45008
Primeiros 5 itens: ['NP:1000075_APAC:3338590_PEE:628646', 'NP:1000075_APAC:3478503_OCM:2916970', 'NP:1000075_APAC:3619209_PEE:823972', 'NP:1000075_APAC:3639720_C:375227_CD:193583', 'NP:1000075_APAC:3672702_C:377880_CD:196223']


In [39]:
# --------------------------------------------
# 5b) PROD (parser only)
# --------------------------------------------
# Gera a lista de PartnerIDs para consulta em prod, a partir do que geramos anteriormente:
partner_ids_str = ",".join([f"'{pid}'" for pid in partner_ids])

# Confirme que a string foi criada
print("Tipo:", type(partner_ids_str))
print("Tamanho:", len(partner_ids_str))

# Veja só o início
print("Início (100 chars):", partner_ids_str[:100])

# Veja o padrão (mostra que está formatada corretamente)
print("Contém aspas simples?", "'" in partner_ids_str)
print("Contém vírgulas?", "," in partner_ids_str)

# Conte quantos IDs tem na string
num_ids_na_string = partner_ids_str.count("'") // 2  # Cada ID tem 2 aspas
print("Número de IDs na string:", num_ids_na_string)

Tipo: <class 'str'>
Tamanho: 1674681
Início (100 chars): 'NP:1000075_APAC:3338590_PEE:628646','NP:1000075_APAC:3478503_OCM:2916970','NP:1000075_APAC:3619209_
Contém aspas simples? True
Contém vírgulas? True
Número de IDs na string: 45008


In [40]:
# 1. Primeiro, crie a tabela temporária com os IDs
ids_df = pd.DataFrame(partner_ids, columns=['PartnerID'])
ids_df.to_sql('temp_partner_ids', engine_prod, if_exists='replace', index=False)


8

In [41]:
# 2. Query para Measurements usando JOIN
query_med_prod = """
SELECT
    m.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    m.Type_ID AS Measurement_ID,
    m.Side,
    m.StringValue,
    m.Metadata,
    mt.Name AS MeasurementName
FROM Measurements m
JOIN EHRs e ON e.ID = m.EHR_ID
JOIN temp_partner_ids t ON e.PartnerID = t.PartnerID
LEFT JOIN MeasurementTypes mt ON m.Type_ID = mt.ID
WHERE m.llmprompt_ID IS NULL
    AND m.OriginType <> 1
"""
df_meas_prod = pd.read_sql(query_med_prod, engine_prod)
print(f"Measurements encontrados: {len(df_meas_prod)}")

# 3. Query para Prescriptions usando JOIN
query_rx_prod = """
SELECT
    m.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    m.Medication_ID AS Measurement_ID,
    m.Side,
    m.Frequency,
    m.FrequencyInstructions,
    m.Duration,
    m.StartDay,
    m.MedicationDosage,
    m.Dosage,
    m.MedicationDosageUnit,
    m.RouteOfAdministration,
    m.Metadata
FROM Prescriptions m
JOIN EHRs e ON e.ID = m.EHR_ID
JOIN temp_partner_ids t ON e.PartnerID = t.PartnerID
WHERE m.llmprompt_ID IS NULL
"""
df_rx_prod = pd.read_sql(query_rx_prod, engine_prod)
print(f"Prescriptions encontradas: {len(df_rx_prod)}")

# 4. Query para Procedures usando JOIN (corrigindo o erro do 'm.llmprompt_ID')
query_proc_prod = """
SELECT
    pe.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    pe.Procedure_ID,
    pe.ProcedureType_ID,
    pt.DisplayName,
    pe.Side,
    pe.PeriodSpan,
    pe.Metadata,
    pr.ProcedureDate,
    pe.DateCreated AS EvidenceCreated
FROM ProcedureEvidences pe
JOIN EHRs e ON pe.EHR_ID = e.ID
JOIN temp_partner_ids t ON e.PartnerID = t.PartnerID
JOIN ProcedureTypes pt ON pe.ProcedureType_ID = pt.ID
LEFT JOIN Procedures pr ON pe.Procedure_ID = pr.ID
WHERE pe.llmprompt_ID IS NULL
"""
df_proc_prod = pd.read_sql(query_proc_prod, engine_prod)
print(f"Procedures encontrados: {len(df_proc_prod)}")

# Visualizar resultados
print("\nPrimeiras linhas de cada tabela:")
print("=== MEASUREMENTS ===")
print(df_meas_prod.head())
print("\n=== PRESCRIPTIONS ===")
print(df_rx_prod.head())
print("\n=== PROCEDURES ===")
print(df_proc_prod.head())

Measurements encontrados: 65859
Prescriptions encontradas: 15843
Procedures encontrados: 4925

Primeiras linhas de cada tabela:
=== MEASUREMENTS ===
    EHR_ID                           PartnerID extractionType  Measurement_ID  \
0  5584556    NP:287847_APAC:2220141_LP:416942         parser            45.0   
1  5521252   NP:926182_APAC:3638534_MR:1627051         parser            45.0   
2  5979663  NP:1114768_APAC:3463244_MR:1531983         parser            45.0   
3  5933270   NP:815374_APAC:3835188_MR:1746906         parser           618.0   
4  6028570   NP:690253_APAC:3705193_MR:1668476         parser           618.0   

   Side StringValue Metadata                      MeasurementName  
0     2       20/40     None  Visual Acuity Snellen No Correction  
1     0      20/100     None  Visual Acuity Snellen No Correction  
2     2      20/200     None  Visual Acuity Snellen No Correction  
3     3     +0.00 D     None                  SphericalEquivalent  
4     3     +0.00 D     

In [20]:
query_med_prod = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    m.Type_ID AS Measurement_ID,
    m.Side,
    m.StringValue,
    m.Metadata,
    mt.Name AS MeasurementName
FROM Measurements m
JOIN EHRs e ON e.ID = m.EHR_ID
LEFT JOIN MeasurementTypes mt ON m.Type_ID = mt.ID
WHERE e.PartnerID IN ({partner_ids_str}) 
    AND m.llmprompt_ID IS NULL
    AND m.OriginType <> 1
"""
df_meas_prod = pd.read_sql(query_med_prod, engine_prod)
df_meas_prod.head()

,EHR_ID,PartnerID,extractionType,Measurement_ID,Side,StringValue,Metadata,MeasurementName
0,5584556,NP:287847_APAC:2220141_LP:416942,parser,44,1,20/40,None,Visual Acuity Snellen
1,6199422,NP:466702_APAC:1547960_MR:409237,parser,44,3,20/20,None,Visual Acuity Snellen
2,6199434,NP:466702_APAC:1363284_MR:306606,parser,44,3,20/20,None,Visual Acuity Snellen
3,5599081,NP:703176_APAC:3905907_MR:1800637,parser,44,2,20/30,None,Visual Acuity Snellen
4,5633703,NP:446310_APAC:3635662_MR:1625007,parser,44,1,20/60,None,Visual Acuity Snellen


In [ ]:
# Prescricoes em prod:
query_rx_prod = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    m.Medication_ID AS Measurement_ID,
    m.Side,
    m.Frequency,
    m.FrequencyInstructions,
    m.Duration,
    m.StartDay,
    m.MedicationDosage,
    m.Dosage,
    m.MedicationDosageUnit,
    m.RouteOfAdministration,
    m.Metadata
FROM Prescriptions m
JOIN EHRs e ON e.ID = m.EHR_ID
WHERE e.PartnerID IN ({partner_ids_str})
    AND m.llmprompt_ID IS NULL

"""
df_rx_prod = pd.read_sql(query_rx_prod, engine_prod)
df_rx_prod.head()

In [ ]:
query_proc_prod = f"""
SELECT
    pe.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    pe.Procedure_ID,
    pe.ProcedureType_ID,
    pt.DisplayName,
    pe.Side,
    pe.PeriodSpan,
    pe.Metadata,
    pr.ProcedureDate,
    pe.DateCreated AS EvidenceCreated
FROM ProcedureEvidences pe
JOIN EHRs e ON pe.EHR_ID = e.ID
JOIN ProcedureTypes pt ON pe.ProcedureType_ID = pt.ID
LEFT JOIN Procedures pr ON pe.Procedure_ID = pr.ID
WHERE e.PartnerID IN ({partner_ids_str})
    AND m.llmprompt_ID IS NULL

"""
df_proc_prod = pd.read_sql(query_proc_prod, engine_prod)
df_proc_prod.head()

In [42]:
# Unificar medidas, prescrições e procedimentos
df_meas_all = pd.concat([df_meas_stg, df_meas_prod], ignore_index=True, sort=False)
df_rx_all = pd.concat([df_rx_stg, df_rx_prod], ignore_index=True, sort=False)
df_proc_all = pd.concat([df_proc_stg, df_proc_prod], ignore_index=True, sort=False)


In [43]:
def dedup_parser_rows(df: pd.DataFrame,
                      id_col: str,
                      name_col: str,
                      value_col: str,
                      etype_col: str,
                      ehr_col: str = "EHR_ID",
                      metadata_col: str = "Metadata",
                      side_col: str | None = None) -> pd.DataFrame:
    """
    Deduplica *apenas* registros do parser dentro do mesmo EHR,
    mantendo 1 por (EHR_ID, ID, Name, Value[, Side]), preferindo a linha com Metadata.
    """
    if df is None or df.empty:
        return df

    required = [id_col, name_col, value_col, etype_col, ehr_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"Faltam colunas para dedupe: {missing}")

    d = df.copy()
    mask_parser = d[etype_col].astype(str).str.lower().eq("parser")
    pars = d.loc[mask_parser].copy()
    other = d.loc[~mask_parser].copy()

    def _s(x):  # string canon básica
        return ("" if pd.isna(x) else str(x)).strip()

    def _s_intlike(x):  # ex.: 2.0 -> '2'
        v = pd.to_numeric(x, errors="coerce")
        return "" if pd.isna(v) else str(int(v))

    # monta chave
    key_parts = [
        pars[ehr_col].astype("Int64").astype(str),
        pars[id_col].astype("Int64").astype(str),
        pars[name_col].map(_s),
        pars[value_col].map(_s),
    ]
    if side_col and side_col in pars.columns:
        key_parts.append(pars[side_col].map(_s_intlike))

    pars["_k"] = "||".join(key_parts) if isinstance(key_parts, str) else (
        key_parts[0] + "||" + key_parts[1] + "||" + key_parts[2] + "||" + key_parts[3] +
        (("||" + key_parts[4]) if len(key_parts) == 5 else "")
    )

    # preferir quem tem metadata não vazia
    pars["_keep_pref"] = pars.get(metadata_col, "").astype(str).str.len().gt(0)

    # ordena e deduplica
    pars = pars.sort_values(by=["_k", "_keep_pref"], ascending=[True, False])
    pars = pars.drop_duplicates(subset=["_k"], keep="first").drop(columns=["_k", "_keep_pref"])

    return pd.concat([other, pars], ignore_index=True, sort=False)


In [44]:
df_meas_all = dedup_parser_rows(
    df_meas_all,
    id_col="Measurement_ID",
    name_col="MeasurementName",
    value_col="StringValue",
    etype_col="extractionType",
    ehr_col="EHR_ID",
    metadata_col="Metadata",
    side_col="Side"
)


In [45]:
# Coagir Measurement_ID para inteiro quando possível (corrige LLM string vs int)
for _df in (df_meas_all, df_rx_all):
    _df["Measurement_ID"] = pd.to_numeric(_df["Measurement_ID"], errors="coerce").astype("Int64")

In [ ]:
df_meas_all.columns.tolist()

In [ ]:
df_rx_all.columns.tolist()

In [ ]:
df_proc_all.columns.tolist()

In [46]:
# ------- utilidades -------
COMMON_COLS = [
    "PartnerID", "EHR_ID", "Group", "ExtractionType",
    "Item_ID", "Item_Name", "Side",
    "Value", "Units",
    "Frequency", "FrequencyInstructions", "Duration", "StartDay", "RouteOfAdministration",
    "PeriodSpan", "ProcedureDate",
    "Metadata", "Details"   # Details: dict/JSON com extras específicos de cada grupo
]

def _safe_col(df, col, default=None):
    return df[col] if col in df.columns else default

def _ensure_cols(df, cols):
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df

def _to_jsonable(d):
    # Remove chaves com NA/None para ficar enxuto
    return {k: v for k, v in d.items() if pd.notna(v) and v is not None}

def normalize_procedures(df_proc_all):
    if df_proc_all is None or len(df_proc_all) == 0:
        return pd.DataFrame(columns=COMMON_COLS)

    out = pd.DataFrame({
        "PartnerID": _safe_col(df_proc_all, "PartnerID"),
        "EHR_ID": _safe_col(df_proc_all, "EHR_ID"),
        "Group": "procedure",
        "ExtractionType": _safe_col(df_proc_all, "extractionType"),
        # >>> AQUI: usar ProcedureType_ID como chave única p/ mapeamento <<<
        "Item_ID": _safe_col(df_proc_all, "ProcedureType_ID"),
        "Item_Name": _safe_col(df_proc_all, "DisplayName"),
        "Side": _safe_col(df_proc_all, "Side"),
        "Value": pd.NA,
        "Units": pd.NA,
        "Frequency": pd.NA,
        "FrequencyInstructions": pd.NA,
        "Duration": pd.NA,
        "StartDay": pd.NA,
        "RouteOfAdministration": pd.NA,
        "PeriodSpan": _safe_col(df_proc_all, "PeriodSpan"),
        "ProcedureDate": _safe_col(df_proc_all, "ProcedureDate"),
        "Metadata": _safe_col(df_proc_all, "Metadata"),
    })
    # Preserve Procedure_ID e outros campos em Details
    details = []
    for _, r in df_proc_all.iterrows():
        d = {
            "Procedure_ID": r.get("Procedure_ID", pd.NA),
            "ProcedureType_ID": r.get("ProcedureType_ID", pd.NA),
            "EvidenceCreated": r.get("EvidenceCreated", pd.NA),
        }
        details.append(_to_jsonable(d))
    out["Details"] = details
    return _ensure_cols(out, COMMON_COLS)

# ------- Normalizações por grupo -------
def normalize_measurements(df_meas_all):
    if df_meas_all is None or len(df_meas_all) == 0:
        return pd.DataFrame(columns=COMMON_COLS)

    out = pd.DataFrame({
        "PartnerID": _safe_col(df_meas_all, "PartnerID"),
        "EHR_ID": _safe_col(df_meas_all, "EHR_ID"),
        "Group": "measurement",
        "ExtractionType": _safe_col(df_meas_all, "extractionType"),
        "Item_ID": _safe_col(df_meas_all, "Measurement_ID"),
        "Item_Name": _safe_col(df_meas_all, "MeasurementName"),   # pode ser NA
        "Side": _safe_col(df_meas_all, "Side"),
        "Value": _safe_col(df_meas_all, "StringValue"),
        "Units": pd.NA,  # se tiver Units em algum lugar, mapeie aqui
        "Frequency": pd.NA,
        "FrequencyInstructions": pd.NA,
        "Duration": pd.NA,
        "StartDay": pd.NA,
        "RouteOfAdministration": pd.NA,
        "PeriodSpan": pd.NA,
        "ProcedureDate": pd.NA,
        "Metadata": _safe_col(df_meas_all, "Metadata"),
    })
    # Extras específicos de measurements (se quiser guardar mais coisas aqui no futuro)
    details = []
    for _, r in df_meas_all.iterrows():
        d = {}
        # exemplo: se houver colunas adicionais em df_meas_all que queira preservar, adicione aqui
        details.append(_to_jsonable(d))
    out["Details"] = details
    return _ensure_cols(out, COMMON_COLS)

def normalize_prescriptions(df_rx_all):
    if df_rx_all is None or len(df_rx_all) == 0:
        return pd.DataFrame(columns=COMMON_COLS)

    out = pd.DataFrame({
        "PartnerID": _safe_col(df_rx_all, "PartnerID"),
        "EHR_ID": _safe_col(df_rx_all, "EHR_ID"),
        "Group": "prescription",
        "ExtractionType": _safe_col(df_rx_all, "extractionType"),
        "Item_ID": _safe_col(df_rx_all, "Measurement_ID"),  # aqui é Medication_ID na origem
        "Item_Name": pd.NA,  # você não tem MedicationName, então fica NA
        "Side": _safe_col(df_rx_all, "Side"),
        "Value": _safe_col(df_rx_all, "Dosage"),            # escolha principal para "Value"
        "Units": _safe_col(df_rx_all, "MedicationDosageUnit"),
        "Frequency": _safe_col(df_rx_all, "Frequency"),
        "FrequencyInstructions": _safe_col(df_rx_all, "FrequencyInstructions"),
        "Duration": _safe_col(df_rx_all, "Duration"),
        "StartDay": _safe_col(df_rx_all, "StartDay"),
        "RouteOfAdministration": _safe_col(df_rx_all, "RouteOfAdministration"),
        "PeriodSpan": pd.NA,
        "ProcedureDate": pd.NA,
        "Metadata": _safe_col(df_rx_all, "Metadata"),
    })
    # Preserve outros campos relevantes em Details
    details = []
    for _, r in df_rx_all.iterrows():
        d = {
            "MedicationDosage": r.get("MedicationDosage", pd.NA),
            # Adicione aqui o que mais quiser preservar em JSON
        }
        details.append(_to_jsonable(d))
    out["Details"] = details
    return _ensure_cols(out, COMMON_COLS)


In [47]:
# ------- Execução: normalizar e concatenar -------
df_meas_norm = normalize_measurements(df_meas_all)
df_rx_norm   = normalize_prescriptions(df_rx_all)
df_proc_norm = normalize_procedures(df_proc_all)


In [48]:
df_meas_norm = dedup_parser_rows(
    df_meas_norm,
    id_col="Item_ID",
    name_col="Item_Name",
    value_col="Value",
    etype_col="ExtractionType",
    ehr_col="EHR_ID",
    metadata_col="Metadata",
    side_col="Side"
)


In [49]:
# --- Deduplicar apenas MEASUREMENTS do PARSER por (EHR_ID, Item_ID, Side, Value) ---
mask = (df_meas_norm["Group"].eq("measurement")) & (df_meas_norm["ExtractionType"].eq("parser"))

meas_parser = df_meas_norm.loc[mask].copy()

# normalizar Side/Value para evitar duplicatas por espaços/NaN
meas_parser["Side"]  = meas_parser["Side"].fillna("").astype(str).str.strip()
meas_parser["Value"] = meas_parser["Value"].fillna("").astype(str).str.strip()

meas_parser = meas_parser.drop_duplicates(
    subset=["EHR_ID", "Item_ID", "Side", "Value"], keep="first"
)

# recompor df_meas_norm com parser deduplicado
df_meas_norm = pd.concat([df_meas_norm.loc[~mask], meas_parser], ignore_index=True)


In [50]:
# 2) União em um único dataframe “longo”
df_all_measures = pd.concat(
    [df_meas_norm, df_rx_norm, df_proc_norm],
    ignore_index=True, sort=False
)

C:\Users\dxr1276\AppData\Local\Temp\ipykernel_15556\1642647056.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_measures = pd.concat(


In [51]:
# df_all_measures: usar Item_ID como chave
df_all_measures = df_all_measures.copy()
df_all_measures["Item_ID"] = pd.to_numeric(df_all_measures["Item_ID"], errors="coerce").astype("Int64")


In [52]:
# Carregar expectativas
with open("mapa_expectativas_nova.json", "r", encoding="utf-8") as f:
    expectations = json.load(f)

df_expect = pd.DataFrame(expectations)

def maybe_json(x):
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return x
    return x

df_expect["possible_ids"] = df_expect["possible_ids"].apply(maybe_json)

while df_expect["possible_ids"].apply(lambda v: isinstance(v, list)).any():
    df_expect = df_expect.explode("possible_ids", ignore_index=True)

In [53]:
# 0) Garantir nome do tipo
df_expect = df_expect.rename(columns={"type": "ClassificationType"})

# 1) Coagir possible_ids para numérico (placeholders viram NaN)
df_expect["possible_id"] = pd.to_numeric(df_expect["possible_ids"], errors="coerce").astype("Int64")

# 2) Tabela de mapeamento por ID (para join com medidas/Rx/procedures)
df_expect_ids = (
    df_expect
    .dropna(subset=["possible_id"])
    .loc[:, ["ClassificationType", "possible_id"]]
    .drop_duplicates()
)

# 3) Tabela de metas por tipo + lista FLAT de IDs
#    (reconstituímos a lista a partir do df_expect_ids para garantir que está achatada)
#    Primeiro, pegamos só os limites (min/med/max) do JSON original (não explodido):
df_limits = (
    pd.DataFrame(expectations)
    .rename(columns={"type": "ClassificationType"})
    [["ClassificationType", "min_expected_measurement", "median_expected_measurement", "max_expected_measurement"]]
)

#    Agora, agregamos os IDs únicos por tipo:
ids_flat = (
    df_expect_ids
    .groupby("ClassificationType", as_index=False)["possible_id"]
    .agg(lambda s: sorted([int(x) for x in s.dropna().unique()]))
    .rename(columns={"possible_id": "possible_ids"})
)

#    Por fim, juntamos limites + ids (tipos sem IDs ficam com lista vazia):
df_expect_meta = df_limits.merge(ids_flat, on="ClassificationType", how="left")
df_expect_meta["possible_ids"] = df_expect_meta["possible_ids"].apply(lambda v: v if isinstance(v, list) else [])
df_expect_meta.head()

,ClassificationType,min_expected_measurement,median_expected_measurement,max_expected_measurement,possible_ids
0,DiagnosisOphthalmological,0,0,0,[]
1,DiagnosisComorbidities,0,0,0,[]
2,FamiliarHistory,0,0,0,[]
3,PreviousOphthalmologicalProcedures,1,2,4,"[6, 7, 8, 9, 10, 11, 12, 14, 16, 19, 20, 21, 2..."
4,MedicationsInUse,1,2,4,"[8345, 8349, 8350, 8351, 8353, 8373, 9280, 932..."


In [54]:
print("#linhas em df_expect_ids:", len(df_expect_ids))
print("Exemplos de tipos e contagens de IDs:")
print(df_expect_ids["ClassificationType"].value_counts().head(20))

# Conferir se ainda sobrou algo não numérico (não deve):
sobras = df_expect.loc[df_expect["possible_id"].isna(), ["ClassificationType", "possible_ids"]].head(20)
print("Exemplos de 'possible_ids' não numéricos (ignorados no join):\n", sobras)


#linhas em df_expect_ids: 1006
Exemplos de tipos e contagens de IDs:
ClassificationType
PreviousOphthalmologicalProcedures    390
SurgicalDescription                   390
MedicationsInUse                       61
MedicationsPrescribed                  61
Refraction                             42
RefractionOnly                         41
VisualAcuity                           10
IntraocularPressure                     9
Cid                                     2
Name: count, dtype: int64
Exemplos de 'possible_ids' não numéricos (ignorados no join):
                    ClassificationType possible_ids
0           DiagnosisOphthalmological          NaN
1              DiagnosisComorbidities          NaN
2                     FamiliarHistory          NaN
515                          SlitLamp          NaN
516   NeurossensorialOcularEvaluation          NaN
580                    MedicalConduct          NaN
622                 ExaminationReport          NaN
1013                 AnesthesiaRecord

In [55]:
# Unificar extrações normalizadas
df_all_measures = pd.concat([df_meas_norm, df_rx_norm, df_proc_norm], ignore_index=True, sort=False)

# Garantir tipagem da chave
df_all_measures["Item_ID"] = pd.to_numeric(df_all_measures["Item_ID"], errors="coerce").astype("Int64")

# Join por ID para obter a ClassificationType de cada linha extraída
measures_with_class = df_all_measures.merge(
    df_expect_ids.rename(columns={"possible_id": "Item_ID"})[["ClassificationType", "Item_ID"]],
    on="Item_ID",
    how="inner"
)


C:\Users\dxr1276\AppData\Local\Temp\ipykernel_15556\3228459044.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_measures = pd.concat([df_meas_norm, df_rx_norm, df_proc_norm], ignore_index=True, sort=False)


In [56]:
# Contagens unificadas (measurement + prescription + procedure)
df_counts_all = (
    measures_with_class
    .groupby(["PartnerID", "ClassificationType", "ExtractionType"])
    .size()
    .reset_index(name="n")
    .pivot(index=["PartnerID", "ClassificationType"], columns="ExtractionType", values="n")
    .reset_index()
    .fillna(0)
    .rename(columns={"llm": "Medidas_llm", "parser": "Medidas_parser"})
)

# Garantir colunas mesmo se alguma origem não aparecer
for c in ["Medidas_llm", "Medidas_parser"]:
    if c not in df_counts_all.columns:
        df_counts_all[c] = 0

# (check rápido)
df_counts_all.head()

ExtractionType,PartnerID,ClassificationType,Medidas_llm,Medidas_parser
0,NP:1000075_APAC:2687830_MR:1048754,PreviousOphthalmologicalProcedures,0.0,8.0
1,NP:1000075_APAC:2687830_MR:1048754,Refraction,10.0,10.0
2,NP:1000075_APAC:2687830_MR:1048754,RefractionOnly,10.0,10.0
3,NP:1000075_APAC:2687830_MR:1048754,SurgicalDescription,0.0,8.0
4,NP:1000075_APAC:2687830_OCM:2151172,IntraocularPressure,4.0,2.0


In [57]:
# base = 1 linha por (EHR, ClassificationType) presente no Contains + metas do JSON
# Pré: ehr_flags tem ["EHR_ID","Patient_ID","PartnerID","EHR_Date","FormattedText","Contains","ClassificationType"]
#      df_expect_meta tem ["ClassificationType","min_expected_measurement","median_expected_measurement",
#                          "max_expected_measurement","possible_ids"]
base = ehr_flags.merge(df_expect_meta, on="ClassificationType", how="left")

# juntar contagens (measurement + prescription + procedure) por PartnerID+ClassificationType
df_final = base.merge(
    df_counts_all, on=["PartnerID","ClassificationType"], how="left"
)

# garantir colunas e tipos
for c in ["Medidas_llm","Medidas_parser"]:
    if c not in df_final.columns: df_final[c] = 0
df_final[["Medidas_llm","Medidas_parser"]] = df_final[["Medidas_llm","Medidas_parser"]].fillna(0).astype(int)

# Total e metas
df_final["Total"] = df_final["Medidas_llm"] + df_final["Medidas_parser"]
for c in ["min_expected_measurement","median_expected_measurement","max_expected_measurement"]:
    df_final[c] = pd.to_numeric(df_final[c], errors="coerce").fillna(0).astype(int)

df_final["Atingiu_minimo"]  = df_final["Total"] >= df_final["min_expected_measurement"]
df_final["Atingiu_mediana"] = df_final["Total"] >= df_final["median_expected_measurement"]
df_final["Atingiu_maximo"]  = df_final["Total"] >= df_final["max_expected_measurement"]

# ‘type’ = alias para compatibilidade e ordenar colunas chave
df_final["type"] = df_final["ClassificationType"]
df_final = df_final.sort_values(["Patient_ID","EHR_ID","ClassificationType"])
df_final.head()


,EHR_ID,Patient_ID,PartnerID,EHR_Date,FormattedText,Contains,ClassificationType,min_expected_measurement,median_expected_measurement,max_expected_measurement,possible_ids,Medidas_llm,Medidas_parser,Total,Atingiu_minimo,Atingiu_mediana,Atingiu_maximo,type
9970,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,DiagnosisComorbidities,0,0,0,[],0,0,0,True,True,True,DiagnosisComorbidities
0,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,DiagnosisOphthalmological,0,0,0,[],0,0,0,True,True,True,DiagnosisOphthalmological
68271,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,MedicalConduct,0,0,0,[],0,0,0,True,True,True,MedicalConduct
60510,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,Refraction,1,6,12,"[52, 53, 54, 55, 56, 57, 58, 475, 476, 477, 48...",6,6,12,True,True,True,Refraction
33571,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,SlitLamp,0,0,0,[],0,0,0,True,True,True,SlitLamp


In [58]:
def to_json_safe(df_rows: pd.DataFrame, cols):
    out = []
    for row in df_rows.to_dict(orient="records"):
        item = {}
        for c in cols:
            if c in row and pd.notnull(row[c]):
                v = row[c]
                # timestamps em ISO
                if isinstance(v, pd.Timestamp):
                    v = v.isoformat()
                item[c] = v
        out.append(item)
    return json.dumps(out, ensure_ascii=False)

# 1) Detalhes de MEASUREMENTS
meas_grouped = (
    df_meas_all.groupby(["PartnerID","extractionType"], dropna=False)
    .apply(lambda g: to_json_safe(
        g,
        ["EHR_ID","extractionType","Measurement_ID","MeasurementName","Side","StringValue","Metadata"]
    ))
    .unstack(fill_value="[]")
    .reset_index()
    .rename(columns={"llm":"measurements_llm","parser":"measurements_parser"})
)

# 2) Detalhes de PRESCRIPTIONS
rx_grouped = (
    df_rx_all.groupby(["PartnerID","extractionType"], dropna=False)
    .apply(lambda g: to_json_safe(
        g,
        ["EHR_ID","extractionType","Measurement_ID","MedicationName","Side",
         "Frequency","FrequencyInstructions","Duration","StartDay",
         "MedicationDosage","Dosage","MedicationDosageUnit","RouteOfAdministration","Metadata"]
    ))
    .unstack(fill_value="[]")
    .reset_index()
    .rename(columns={"llm":"prescriptions_llm","parser":"prescriptions_parser"})
)

# 3) Detalhes de PROCEDURES
proc_grouped = (
    df_proc_all.groupby(["PartnerID","extractionType"], dropna=False)
    .apply(lambda g: to_json_safe(
        g,
        ["EHR_ID","extractionType","Procedure_ID","ProcedureType_ID","DisplayName","Side",
         "PeriodSpan","Metadata","ProcedureDate","EvidenceCreated"]
    ))
    .unstack(fill_value="[]")
    .reset_index()
    .rename(columns={"llm":"procedures_llm","parser":"procedures_parser"})
)

# 4) Mesclar os detalhes no df_final
df_final = (
    df_final
    .merge(meas_grouped, on="PartnerID", how="left")
    .merge(rx_grouped,   on="PartnerID", how="left")
    .merge(proc_grouped, on="PartnerID", how="left")
)

# Garantir colunas e preencher ausências
for c in ["measurements_llm","measurements_parser",
          "prescriptions_llm","prescriptions_parser",
          "procedures_llm","procedures_parser"]:
    if c not in df_final.columns:
        df_final[c] = "[]"
    else:
        df_final[c] = df_final[c].fillna("[]")

print("OK: df_final agora contém as colunas de detalhes JSON.")


OK: df_final agora contém as colunas de detalhes JSON.


In [59]:
df_final.head()

,EHR_ID,Patient_ID,PartnerID,EHR_Date,FormattedText,Contains,ClassificationType,min_expected_measurement,median_expected_measurement,max_expected_measurement,...,Atingiu_minimo,Atingiu_mediana,Atingiu_maximo,type,measurements_llm,measurements_parser,prescriptions_llm,prescriptions_parser,procedures_llm,procedures_parser
0,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,DiagnosisComorbidities,0,0,0,...,True,True,True,DiagnosisComorbidities,"[{""EHR_ID"": 4640894, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6385713, ""extractionType"": ""parser...",[],[],[],[]
1,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,DiagnosisOphthalmological,0,0,0,...,True,True,True,DiagnosisOphthalmological,"[{""EHR_ID"": 4640894, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6385713, ""extractionType"": ""parser...",[],[],[],[]
2,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,MedicalConduct,0,0,0,...,True,True,True,MedicalConduct,"[{""EHR_ID"": 4640894, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6385713, ""extractionType"": ""parser...",[],[],[],[]
3,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,Refraction,1,6,12,...,True,True,True,Refraction,"[{""EHR_ID"": 4640894, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6385713, ""extractionType"": ""parser...",[],[],[],[]
4,4640894,280525,NP:709436_APAC:1636568_OCM:1211163,2017-04-22 11:14:54,Consulta Médica\r\nID da Consulta: 1211163\r\n...,5187,SlitLamp,0,0,0,...,True,True,True,SlitLamp,"[{""EHR_ID"": 4640894, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6385713, ""extractionType"": ""parser...",[],[],[],[]


In [60]:
df_final.to_csv("comparacao_medidas_17092025.csv", index=False)